# Point Pattern Summary Statistics

Created by: Derek Robinson <br>
Last Updated: January 15, 2021

### Learning Objectives

In this Jupyter Notebook you will learn or review how to
<ul>
    <li> load and view shapefile point and polygon data, </li>
    <li> compute spatial summary statistics (mean location, standard distance),</li>
    <li> perform an overlay to retrieve point counts within polygons, </li>
    <li> define the spatial reference system for spatial data generated in R, </li>
    <li> export a SpatialPointsDataFrame to Shapefile, </li>
    <li> conduct spatial analysis using R. </li>
</ul>

### Assignment Completion

For those in GEOG 318/PLAN 353 @UWaterloo, the assignment directions and submission requirements are provided on LEARN as a quiz that spans multiple days. To complete the quiz/assignment you will need to do the following:

<ul>
    <li> Complete this Jupyter Notebook.</li>
    <ul>
        <li> To advance to the next text cell or execute the code in a cell hold <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift</mark></font> and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Enter</mark></font> </li>
        <li> If there is a method for which you would like to know more about its parameters or how it works then you can place your cursor on the method and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift + Tab</mark></font> and a help box will appear that you can expand and scroll through.</li>
    </ul>
    <li> Complete the Spatial Summary Statistics Quiz/Assignment in LEARN, which may require you to </li>
    <ul>
        <li> copy results from your notebook into the LEARN input boxes, </li>
        <li> take screen captures of your notebook output and submit them via LEARN, or </li>
        <li> change values in the Jupyter Notebook and re-execute cells/code to obtain new results. </li>
    </ul>
    <li> <b>While you may discuss course content with your classmates, you are to complete the assignment individually.</b></li>
</ul>


### Problem Statement & Data

The summary statistics calculated below use point pattern data of Home Depot stores located in Ontario, Canada, in 2014. These data along with Census data and other store and spatial data used to conduct analyses for a major home improvement company. The following publication is one output of our retail analysis that can provide some context and illustrate additional types of spatial analysis that can be conducted.

Robinson, D.T., and A. Balulescu, 2018. Comparison of Methods for Quantifying Consumer Spending on Retail
using Publicly Available Data. International Journal of Geographic Information Science, 32(6), 1061-1086. 

**While the methods below use store location data, they are applicable to point data representing other phenomena (e.g., wetland, wildlife, school, or crime locations).** 

### Lets get started

Sometimes when running functions and code in R you will receive warnings. These are displayed in pink in Jupyter Notebook following the execution of a cell. The warnings arise for a variety of reasons like a parameter was not specified so a default will be used. In creating these notebooks I have reviewed the warnings and they do not affect the results. The next line of code turns them off so that you are not confused or worried about them. However, if you would like to see them you may restart the notebook and comment the following cell code or skip executing it.

In [ ]:
options(warn=-1)
# command to turn warnings on
# options(warn=0)

<b>Setup and Reading in Shapefiles</b><br>
In R, we can perform statistical and spatial analyses using shapefile data. Working with these data requires using functions from software packages developed by different groups of people, which are referred to as libraries. 

The first library we need is `rgdal` which contains several functions for handling geo-spatial data. The first time you use a library you have to install it with the `install.packages()` function. After the package is installed you only need to call the `library()` function to gain access to it in a Notebook or R script. In this notebook, the install commands have been commented out using `#` symbols so that the compiler will not interpret the code/text. If you receive an error related to rgdal or other packages then you may need to uncomment the lines and execute the cell again.

In [ ]:
#install.packages("rgdal", repos="http://R-Forge.R-project.org")
library(rgdal)

To import a shapefile we will use the `readOGR()` function from `rgdal`. This function allows us to define a shapefile as a usable object in R. In addition to our SHIFT + TAB shortcut, you can get the details about any function in R by typing a question mark `?` in front of the function at a command line. Try executing the next cell to learn more about the `readOGR` function. Executing the help operator `?` will split the screen and provide the help in another frame. To remove the split screen you will have to close the frame with the `x` in the upper right of the new frame.

In [ ]:
?readOGR

`readOGR()` has two required inputs. The first input is the data source name (dsn) of the file. For us this will be `OntarioHomeDepot.shp`. Since we set the working directory `setwd()` above, we can just use the filename of the file we want to work with as the dsn. If you do not use `setwd()` earlier then the full path and filename must be used.

The second input requirement is layer, which is the layer name `OntarioHomeDepot`. Note that the specifications for dsn and layer may vary with the rgdal function being used, but for now we can use them as text descriptors of our file. I stored my data in a different folder so I specified the location of the shapefile relative to the location of the script using `Data/OntarioHomeDepot.shp`. Data specifies the folder where my data is located. Then I can import the OntarioHomeDepot.shp file using:

In [ ]:
OHD <- readOGR("OntarioHomeDepot.shp", "OntarioHomeDepot")

When working with functions that require parameters or variable inputs you can be more explicit about the inputs by using keyword declarations like dsn= and layer= in front the inputs as follows: 

`OHD <- readOGR(dsn="Data/OntarioHomeDepot.shp", layer="OntarioHomeDepot")`

This can be a bit annoying, but it can be very helpful if you’re curious about the type of input parameter or variable and how it links to the performing of the function. It is also useful if you want to share your code or you do not want to forget about how the code works when you come back at a later date.

Repeat the above step to import the OntarioProvince shapefile into a new object called ONP. 

In [ ]:
ONP <- readOGR(dsn="OntarioProvince.shp", layer="OntarioProvince")

Now we can look at the contents of the objects we created by importing the shapefile. 

R stores data in what are referred to as a `data.frame` and our vector data (i.e., points for stores and polygon for the province) are stored in a type of data frame called a `spatial data frame`. 

Data frames are often thought of as being similar to tables of data. If you type `print (OHD)` and execute the command then the data frame or table representing the Ontario Home Depot shapefile will print out. Note, columns can be split because the console or output window may not be large enough to display all the data. You should also notice that to the left of the first column are the coordinates of each point in parentheses.


In [ ]:
print (OHD)

### Calculating Mean Centre
Simple summary statistics about point patterns can be calculated for spatial data in a similar way to how we calculate summary statistics for aspatial data (e.g., mean, standard deviation).  

Here we will calculate the mean x and y coordinates of our point pattern.

Calculate the “Mean X” and “MeanY” by first extracting x and y coordinate values from the “X” and “Y” columns, the last two columns of the table. We can do this by using the `$` method. This involves first writing the name of the object, then a `$`, and lastly the name of the column. It can be used to select the x coordinate vector (column) and place it in a new object of just the x coordinate values as follows:

In [ ]:
OHD_X <- OHD$X 

If we print the `OHD_X` object you'll now see all the x coordinates for all stores. Lets also print out the length of the OHD_X object to confirm that there are the same number of x coordinates in this object are there are stores, which we can see above when we used the `print(OHD)` command.

In [ ]:
print (OHD_X)
print (length(OHD_X))

Next, use the `mean()` function to derive the mean of the x-coordinate values found in your OHD_X vector.

In [ ]:
OHD_X_mean <- mean(OHD_X)

You can look at your result by typing the variable name (i.e, `OHD_X_mean`) in the `print()` command. However, in this case it may surprise you with no decimal places for the x-coordinates. Printing values with R can end up with rounding done for you, whether you want it or not. Therefore, it is always best to identify the number of significant digits you would like in your output. We can specify the number of digits after the decimal using the `format()` function. which takes the following form `(<number>, nsmall=<number of digits>)`.
To view the difference, execute the next two cells.

In [ ]:
print (OHD_X_mean)

In [ ]:
print (format(OHD_X_mean, nsmall=2))

In the last command, you nested the format function inside the print function and upon viewing your results you should have seen two decimal points. If you would like to add additional descriptive text, then you can use the paste() command and alter the previous line of code above as:

In [ ]:
print (paste("x coordinate mean = ", format(OHD_X_mean, nsmall=2)))

In the Jupyter cell below, type in the command to derive the y-coordinate mean and assign it (just the number) to a variable named `OHD_Y_mean`

In [ ]:
OHD_Y_mean <- mean(OHD$Y)

### Visualizing the Mean Centre
Now that we have created objects that hold the mean of the x and y coordinates, we can visualize the mean centre by creating a new spatial point object.  

Our first step is to create a new `data.frame` and initialize that data frame with our mean x and y values

In [ ]:
meanCentre <- data.frame(OHD_X_mean, OHD_Y_mean)

Print out the `meanCentre` object to see the contents of the data frame.  

In [ ]:
print (meanCentre)

What you should see is that `OHD_X_mean` and `OHD_Y_mean` are columns in the data frame, which only has one row since each of these variables contains only a single value. If OHD_X_mean and OHD_Y_mean had been vectors, then the data frame would have multiple rows. Since we just created the meanCentre dataframe from two objects, its type is different from the OHD and ONP dataframes. Execute the `class()` function to observe the type of data frame we just created.

In [ ]:
class(meanCentre)

Type and execute the `class()` function to observe the type of dataframe for the `OHD` and `ONP` objects.

In [ ]:
class(OHD)

To create spatially usable coordinates for our mean point location, we need to use a function from the package `sp` called `coordinates()`. Let's load the sp library using the same process we used for rgdal above.

In [ ]:
#install.packages("sp")
library(sp)

We can use the coordinates function to print existing coordinates or embed new coordinates. Try using `print(coordinates(OHD))` to view the coordinates of the points in your OHD data frame. 

In [ ]:
print(coordinates(OHD))

To convert the coordinate values for our mean centre to a spatial coordinate we will use the `sp` package `coordinates()` function. 

In [ ]:
coordinates(meanCentre)

Next, visualize the spatial distribution of the individual Home Depot stores and the mean centre of the store point distribution. To do this we could use the `plot()` command, which is a simple command to view the data. However, even with our data it can take a long time to execute and it is hard to use to create good looking maps.

Instead we will use a few functions from other packages to display our map. I will not go into detail about the code for map display nor will you be required to create your own code or be tested on map display code. However, you will need to be able to execute it to prove the output of your statistical work above as part of your assignment submission.

<b>Be patient as it may take a minute or two for the map to render and be displayed</b>

In [ ]:
#install.packages("ggplot2")
#install.packages("ggmap")
#install.packages("sf")
#install.packages("ggspatial")
#install.packages("patchwork")

library(ggplot2)
library(ggmap)
library(sf)
library(ggspatial)
library(patchwork)

OHD_data <- data.frame(OHD)
ONP_data <- fortify(ONP)
meanCentre_data <- data.frame(meanCentre)

g <- ggplot() + geom_polygon(data=ONP_data, aes(x=long, y=lat, group=group), color="black", fill=NA)

plot(g)

Now we will add the store point data on top of the provincial layer.

<b>You can right click on any of the output plots and save them as an image for submission in your assignment.</b> <br>You can also open the image in a new tab for viewing purposes.

In [ ]:
g1 <- g + geom_point(data=OHD_data, aes(x=X, y=Y), color="red")
plot(g1)

Now we will add the `meanCentre` in blue and clean up the labels on the x and y axes as well as add a scale bar.

In [ ]:
g2 <- g1 + geom_point(data=meanCentre_data, aes(x=OHD_X_mean, y=OHD_Y_mean), colour="blue") + coord_sf()
g3 <- g2 + xlab("Longitude") + ylab("Latitude") + ggtitle("OHD and Mean Centre") + annotation_scale(location = "bl", width_hint = 0.5)
g4 <- g3 + annotation_scale(location = "bl", width_hint = 0.5)
plot(g4)

In the above plot, you should see the mean store location just north of Toronto. 

### Calculating Standard Distance

Now that we have calculated the mean point location of our point pattern, we are also interested in the spread of the point distribution around that mean location. We can summarize this spread by calculating the standard distance.
To calculate the standard distance of the spread of a spatial point pattern we can use the following equation:

![image](StandardDistanceEquation.png)

where `d` is the distance, <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">(x<sub>i</sub>-μ<sub>x</sub>)<sup>2</sup></mark></font> is the squared x-coordinate residuals, <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">(x<sub>i</sub>-μ<sub>Y</sub>)<sup>2</sup></mark></font> is the squared y-coordinate residuals, the numerator is the sum of these squared coordinate residuals and the denominator `n` is the number of features (i.e., number of coordinate locations or points), then the result of the sum of the squared residuals is divided by n and the square root of that result is taken. You can review calculations of standard deviation from ENV 278 for comparison. 

You can determine the number of features (n) in OHD by using the `length()` function, which returns a count of the number of rows in the OHD data frame. To compute the square root of a value you can use the `sqrt()` function. 

Next, we will <b>calculate the sum of squared residuals</b>. First we need to calculate the squared residuals of our x and y coordinates. The residual is the difference between a value and the mean. A squared residual is that value squared (i.e., residual<sup>2</sup>). First calculate the residuals of the x-coordinates from the mean x-coordinate and then square them <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">(x- μ)<sup>2</sup></mark></font>. Use the next codeblock. Remember that the values in `OHD_X` is a vector of x-coordinates that correspond to the x-coordinates for each store in our data. Therefore, when you subtract the mean of our x-coordinate locations `OHD_X_mean` you are subtracting the mean value from 89 different x-coordinates and storing a new vector of 89 results in the object `xRes`.

In [ ]:
xRes <- (OHD_X - OHD_X_mean) 
xResSqrd <- xRes^2

To acquire the sum of the squared X residuals you can use the `sum()` function as:

In [ ]:
xSumSqrdRes <- sum(xResSqrd)

In the next code cell, create the code required to print out the sum of the squared residuals for the x coordinates up to two significant digits. You can look back in this notebook at the `format()` function for assistance. 

In [ ]:
print (paste("Sum of the squared x-coordinate residuals = ", format(xSumSqrdRes, nsmall=2)))

We have completed the steps for deriving the sum of the squared x-coordinate residuals. In the Jupyter cell below, create the code to calculate the sum of squared y-coordinate residuals. 

In [ ]:
OHD_Y_mean <- mean(OHD$Y)
yRes <- (OHD$Y - OHD_Y_mean) 
yResSqrd <- yRes^2
ySumSqrdRes <- sum(yResSqrd)
print (ySumSqrdRes)

#or you could do
ySumSqrdRes <- sum((OHD$Y - mean(OHD$Y))^2)
print (ySumSqrdRes)


Now that we have the sum of the squared x-coordinate and y-coordinate residuals, we can <b>calculate the standard distance</b> using the standard distance equation described above. This involves adding the sum of the squared residuals for the x and y coordinates together. Dividing that result by the number of points in our pattern `n` and then taking the square root `sqrt()` of the result.

In [ ]:
n <- length(OHD$X)
stdDist <- sqrt((xSumSqrdRes + ySumSqrdRes)/n)
print(stdDist)

The standard distance we just calculated is for both x and y directions. Displaying a standard distance around our mean centre point would produce a circle. It is possible to calculate the standard distance for each coordinate direction (i.e., x and y) so that a more accurate representation is achieve using an ellipse. For now, we will identify the distance at 1, 2, and 3 standard distances from the mean and assign them each to a new variable.

In [ ]:
stdist1 <- stdDist
stdist2 <- stdDist * 2
stdist3 <- stdDist * 3
print (paste("1 Standard Distance = ", format(stdist1, nsmall=2),", 2 Standard Distances = ", format(stdist2, nsmall=2),", 3 Standard Distances = ", format(stdist3, nsmall=2)))

### Defining Spatial Reference Information
Before we can visualize the standard distance values in a map. We have to define the spatial reference information for our meanCentre object. Currently, meanCentre is just a data frame. However, if we would like to do spatial processes then we must convert it to a spatial data frame. We can do this easily using the `SpatialPointsDataFrame()` function from the `sp` package, which we imported earlier. 

The `SpatialPointsDataFrame()` function requires spatial reference information as an input parameter as it will assign it to the new object created. Use the `proj4string()` function from the `sp` package to extract the spatial reference information from our store points shapefile and put it in a new object `OHD_proj`.

In [ ]:
OHD_proj <- proj4string(OHD)

Now create a new object `meanCentreSpdf` and assign it the output from the `SpatialPointsDataFrame()` function. The function requires a set of coordinates, the data, and the spatial reference information. 

In [ ]:
meanCentreSpdf <- SpatialPointsDataFrame(coords = coordinates(meanCentre), data = meanCentre, proj4string = CRS(OHD_proj))

To verify that the new object is a spatial data frame. Execute the `class()` function in the next cell on our `meanCentreSpdf` object.

In [ ]:
class(meanCentreSpdf)

Now that we have a `SpatialPointsDataFrame` of our meanCentre location, we can use the `gBuffer()` function from the `rgeos` package to create buffers around our mean centre at each of the three standard distances. First, load the `rgeos` package as we have done earlier for rgdal and sp. 

In [ ]:
#install.packages("rgeos")
library(rgeos)

The `gBuffer()` function has only one required input, a spatial data frame object. However, we will also specify the optional width input as the radius (not the diameter, as the word “width” may suggest), which in our case will be our standard distance values. <b>Create and plot standard distance buffers</b> for the three standard distances by executing the following code in the next Jupyter cell.

In [ ]:
stdist1_buf <- gBuffer(meanCentreSpdf, width=stdist1)
stdist2_buf <- gBuffer(meanCentreSpdf, width=stdist2)
stdist3_buf <- gBuffer(meanCentreSpdf, width=stdist3)

stdist1_buf_data <- fortify(stdist1_buf)
stdist2_buf_data <- fortify(stdist2_buf)
stdist3_buf_data <- fortify(stdist3_buf)

std_g1 <- g + geom_polygon(data=stdist1_buf_data, aes(x=long, y=lat, group=group), color="black", fill=NA) +
  geom_polygon(data=stdist2_buf_data, aes(x=long, y=lat, group=group), color="black", fill=NA) +
  geom_polygon(data=stdist3_buf_data, aes(x=long, y=lat, group=group), color="black", fill=NA) +
  xlab("Longitude") + ylab("Latitude") + ggtitle("Standard Distances (1-3)") + annotation_scale(location = "bl", width_hint = 0.5)
plot(std_g1)

### Variation From the Mean
When we calculate the standard deviation of aspatial data, one standard deviation from the mean captures ~68% of the variation from the mean (or approximately 68% of the data). Two standard deviations captures ~95% of the data and three standard deviations capture ~99.7% of the data. Let’s see if this holds for spatial data.

The standard deviation in this case is our standard distance, which we just calculated above. We will use our standard distance calculation to determine the  percent of stores that fall within 1, 2 and 3 standard distances from the mean store location. 

Note that the coordinate system we are using is in metres so the standard distance we have calculated is also in metres. 

Use the `over()` function from the `sp` package to determine if there is an intersection between points and a polygon layer. The `over()` function requires two spatial data frames as inputs. The function returns a vector object with a value of 1 where there is an overlap between the point and the polygon layers and a string variable “NA” where the point does not overlap the polygon. <br><br>
Execute the next cell to identify the stores within the first standard distance.

In [ ]:
stdist1_HD_v <- over(OHD, stdist1_buf)
print(stdist1_HD_v)

To determine the number of stores that intersected our first standard distance buffer we can sum the vector as we did earlier when computing the sum of the squared differences between the coordinates and the mean coordinate. However, this time the vector has NA elements, which represent not available and are string elements and they will create an error. Luckily we can still use the `sum()` function by specifying a parameter to remove NA values as follows:

In [ ]:
stdist1_HD_c <- sum(stdist1_HD_v, na.rm = TRUE)
print (stdist1_HD_c)

Having printed your results you can visually count the 1 values in the vector output above to ensure the sum function is working correctly. Use the `length()` function we used earlier to acquire a count of the number of features in the `OHD` dataframe and divide the number of features in the first standard distance by the total number of features in the dataframe to obtain the proportion of features within one standard distance of the mean store location. 

In [ ]:
a <- (stdist1_HD_c/length(OHD))
print(format((a), digits=2, nsmall=2))

### Verifying R Calculations in ArcGIS
One way to verify that your mean and standard distance calculations are correct is to replicate the process in a GISystem like ArcGIS. In ArcGIS, open ArcToolbox and select Spatial Statistics Tools -> Measuring Geographic Distributions -> Mean Center. In the Mean Center window select OntarioHomeDepot as your input feature class. For your Output Feature Class, navigate to your data folder where OntarioHomeDepot is and call this new file ON_Mean_HDTool so that we know this is the mean location calculated using the ArcGIS tool. Ignore the other optional parameters for this tool and click OK at the bottom of the window. Compare it to your “meanCentre” to ensure that your R mean centre is correct.

Next use ArcGIS to calculate the standard distance for you in ArcToolbox by selecting Spatial Statistics Tools -> Measuring Geographic Distributions -> Standard Distance. Select OntarioHomeDepot as your input feature class. Navigate to your data folder and specify the output feature class as stdev1Tool and select 1_STANDARD_DEVIATION as the circle size, which will be centered on the mean center point that ArcGIS calculated above using the Mean Center tool. Run the Standard Distance tool two more times, one for each 2_STANDARD_DEVIATION and 3_STANDARD_DEVIATION, and call your new layers stdev2Tool and stdev3Tool.

Open the attribute table for stdev1Tool. The mean x point value and mean y point value should be equal to what you calculated. You should also see that the StdDist matches the standard distance value you calculated. If not, then put on your dective hat and try and identify any errors in this notebook.

### Exporting SpatialPointsDataFrame to Shapefile
To export our R data to a shapefile, or to create a shapefile of our R SpatialPointsDataFrame, we will use the function `writeOGR()` from the `rgdal` package, which has the following four required inputs: 

1. the object we want to convert to a shapefile, which can only be of a spatial data frame type
2. the dsn
3. layer similar to `readOGR()` that we used earlier
4. and the driver, that specifies the type of file to be created, which in our case is ESRI Shapefile. 

We will also use one optional input called `overwrite_layer` and assign it to `TRUE` so that we can rerun the code without getting a 'file already exists' error message. Optional variables are preceded by the name of the variable and an equals sign `overwrite_layer=TRUE` to prevent R from confusing it with other optional variables. Execute the command in the next cell to generate a new shapefile of the mean centre of the store points data.

In [ ]:
writeOGR(meanCentreSpdf, "meanCentre.shp", "meanCentre", driver="ESRI Shapefile", overwrite_layer = TRUE)

We can assign the created shapefile to a new object using the assignment operator `<-`, but at this point we are just going to save it to our hard disk. The `meanCentreSpdf` input to the writeOGR function tells the function what data frame we are going to use. The first string “meanCentre.shp” is the dsn (i.e., data source name) output filename. If I did not specify a working directory then I would want to provide the full path prior to the filename.

Open your new file in your favourite geographic information system (GIS) or at least one you have handy. I opened it up in QGIS (http://www.qgis.org/) since I am working on my MacBook. Open the attribute table and you should see your two x and y attributes. You could also navigate to the shapefile properties and review the spatial referencing information.

# Congratulations!

**You have reached the end of the Spatial Summary Statistics notebook**